In [ ]:
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.style.use(r'PaperDoubleFig.mplstyle')
genesis_bin='/home/vagrant/.local/bin/genesis2-mpi'
from genesis import Genesis

In [ ]:
!which genesis2-mpi

## Steady state solution:
```python
gen = Genesis('template.in')
# undulator
gen['xlamd'] = 0.0186
gen['aw0'] = gen['awd'] = 0.86
gen['nwig'] = int(80/gen['xlamd'])
# electron beam
gen['curpeak'] = 3000
gen['gamma0'] = 12e9/0.511e6
gen['delgam'] = 1.5e-4*gen['gamma0']
gen['rxbeam'] = 1.1973e-05
gen['rybeam'] = 1.0446e-05
gen['emitx'] = gen['emity'] = 0.2e-6
gen['npart'] = 2**14
# radiation
gen['xlamds'] = 2.9384e-11
gen['prad0'] = 0.175e6
gen['zrayl'] = 24
gen['zwaist'] = 0
# mesh
gen['ncar'] = 151
gen['dgrid'] = 100e-6
# focusing
gen['f1st'] = 5
gen['fl'] = 10
gen['quadf'] = 30.067217489485472
gen['dl'] = 10
gen['quadd'] = 30
gen['drl'] = 100
gen['nwig'] -= gen['nwig']%(gen['fl']+gen['dl']+2*gen['drl'])
# simulation
gen['delz'] = 5
gen.run()
gen.output['run_info']
```

## Include unphysical slices
```python
gen1 = Genesis('template.in', genesis_bin=genesis_bin)
gen1.binary_prefixes = ['mpirun', '-n', '44']
# undulator
gen1['xlamd'] = 0.0186
gen1['aw0'] = gen1['awd'] = 0.86
gen1['nwig'] = int(80/gen1['xlamd'])
# electron beam
gen1['curpeak'] = 3000
gen1['gamma0'] = 12e9/0.511e6
gen1['delgam'] = 1.5e-4*gen1['gamma0']
gen1['rxbeam'] = 1.1973e-05
gen1['rybeam'] = 1.0446e-05
gen1['emitx'] = gen1['emity'] = 0.2e-6
gen1['npart'] = 2**14
# radiation
gen1['xlamds'] = 2.9384e-11
gen1['prad0'] = 0e4
gen1['zrayl'] = 24
gen1['zwaist'] = 0
# mesh
gen1['ncar'] = 151
gen1['dgrid'] = 100e-6
# focusing
gen1['f1st'] = 5
gen1['fl'] = 10
gen1['quadf'] = 30.067217489485472
gen1['dl'] = 10
gen1['quadd'] = 30
gen1['drl'] = 100
gen1['nwig'] -= gen1['nwig']%(gen1['fl']+gen1['dl']+2*gen1['drl'])
# simulation
gen1['delz'] = 5
gen1['itdp'] = 1
gen1['shotnoise'] = 1.0
gen1['nslice'] = 44*20
gen1['iotail'] = 1
gen1['ipseed'] = 1
gen1['zsep'] = 4*gen1['delz']
gen1['curlen'] = -gen1['nslice']*gen1['zsep']*gen1['xlamds']
gen1.run()
gen1.output['run_info']
```

## Do not include unphysical slices
```python
gen2 = Genesis('template.in', genesis_bin=genesis_bin)
gen2.binary_prefixes = ['mpirun', '-n', '44']
# undulator
gen2['xlamd'] = 0.0186
gen2['aw0'] = gen2['awd'] = 0.86
gen2['nwig'] = int(80/gen2['xlamd'])
# electron beam
gen2['curpeak'] = 3000
gen2['gamma0'] = 12e9/0.511e6
gen2['delgam'] = 1.5e-4*gen2['gamma0']
gen2['rxbeam'] = 1.1973e-05
gen2['rybeam'] = 1.0446e-05
gen2['emitx'] = gen2['emity'] = 0.2e-6
gen2['npart'] = 2**14
# radiation
gen2['xlamds'] = 2.9384e-11
gen2['prad0'] = 0e4
gen2['zrayl'] = 24
gen2['zwaist'] = 0
# mesh
gen2['ncar'] = 151
gen2['dgrid'] = 100e-6
# focusing
gen2['f1st'] = 5
gen2['fl'] = 10
gen2['quadf'] = 30.067217489485472
gen2['dl'] = 10
gen2['quadd'] = 30
gen2['drl'] = 100
gen2['nwig'] -= gen2['nwig']%(gen2['fl']+gen2['dl']+2*gen2['drl'])
# simulation
gen2['delz'] = 5
gen2['itdp'] = 1
gen2['shotnoise'] = 1.0
gen2['nslice'] = 44*20
gen2['iotail'] = 0
gen2['ipseed'] = 1
gen2['zsep'] = 4*gen2['delz']
gen2['curlen'] = -gen2['nslice']*gen2['zsep']*gen2['xlamds']
gen2.run()
gen2.output['run_info']
```

In [ ]:
gen = Genesis()
gen.load_archive('gen.h5')
gen1 = Genesis()
gen1.load_archive('gen1.h5')
gen2 = Genesis()
gen2.load_archive('gen2.h5')

In [ ]:
plt.plot(gen.output['data']['z'], 1e-9*gen.output['data']['power'][0], label='Steady-State (time-independent)')
plt.plot(gen1.output['data']['z'], 1e-9*np.mean(gen1.output['data']['power'], axis=0), label='Unphysical (time-dependent)')
plt.plot(gen2.output['data']['z'], 1e-9*np.mean(gen2.output['data']['power'], axis=0), label='Physical (time-dependent)')
plt.xlabel('Undulator distance, m')
plt.ylabel('Average power, GW')
plt.xlim([0, 79])
plt.ylim([0, np.max(gen.output['data']['power'][0])*1.01e-9])
plt.legend()
plt.show()

In [ ]:
plt.semilogy(gen.output['data']['z'], gen.output['data']['power'][0], label='Steady-State (time-independent)')
plt.semilogy(gen1.output['data']['z'], np.mean(gen1.output['data']['power'], axis=0), label='Unphysical (time-dependent)')
plt.semilogy(gen2.output['data']['z'], np.mean(gen2.output['data']['power'], axis=0), label='Physical (time-dependent)')
plt.xlabel('Undulator distance, m')
plt.ylabel('Average power, W')
plt.xlim([0, 79])
plt.ylim([1e5, np.max(gen.output['data']['power'][0])*1.1])
plt.legend()
plt.show()

In [ ]:
plt.imshow(gen1.output['data']['power'].T, aspect='auto', origin='lower',
           extent=[0, len(gen1.output['data']['power']),
                  gen1.output['data']['z'][0], gen1.output['data']['z'][-1]])
plt.grid()
plt.xlabel('Slice number')
plt.ylabel('Undulator distance, m')
plt.ylim([35, 77])
plt.show()

In [ ]:
plt.plot(1e-9*gen1.output['data']['power'][:,-1])
#plt.xlabel('Slice number')
plt.ylabel('Power, GW')
plt.xlim([0, len(gen1.output['data']['power'])])
plt.ylim([0, np.max(1e-9*gen1.output['data']['power'][:,-1])*1.01])
plt.title('Power with unphysical slices')
plt.show()

In [ ]:
plt.imshow(gen2.output['data']['power'].T, aspect='auto', origin='lower',
           extent=[0, len(gen2.output['data']['power']),
                  gen2.output['data']['z'][0], gen2.output['data']['z'][-1]])
plt.grid()
plt.xlabel('Slice number')
plt.ylabel('Undulator distance, m')
plt.ylim([35, 77])
plt.show()

In [ ]:
plt.plot(1e-9*gen2.output['data']['power'][:,-1])
#plt.xlabel('Slice number')
plt.ylabel('Power, GW')
plt.xlim([0, len(gen2.output['data']['power'])])
plt.ylim([0, np.max(1e-9*gen2.output['data']['power'][:,-1])*1.01])
plt.title('Power with physical slices')
plt.show()

In [ ]:
gen1g = Genesis()
gen1g.load_archive('gen1g.h5')
gen2g = Genesis()
gen2g.load_archive('gen2g.h5')

In [ ]:
plt.imshow(gen1g.output['data']['power'].T, aspect='auto', origin='lower',
           extent=[0, len(gen1g.output['data']['power']),
                  gen1g.output['data']['z'][0], gen1g.output['data']['z'][-1]])
plt.grid()
plt.xlabel('Slice number')
plt.ylabel('Undulator distance, m')
plt.ylim([35, 77])
plt.show()

In [ ]:
plt.plot(1e-9*gen1g.output['data']['power'][:,-1])
#plt.xlabel('Slice number')
plt.ylabel('Power, GW')
plt.xlim([0, len(gen1g.output['data']['power'])])
plt.ylim([0, np.max(1e-9*gen1g.output['data']['power'][:,-1])*1.01])
plt.title('Power with unphysical slices')
plt.twinx()
plt.plot(gen1g.output['data']['current'], 'orange')
plt.show()

In [ ]:
plt.imshow(gen2g.output['data']['power'].T, aspect='auto', origin='lower',
           extent=[0, len(gen2g.output['data']['power']),
                  gen2g.output['data']['z'][0], gen2g.output['data']['z'][-1]])
plt.grid()
plt.xlabel('Slice number')
plt.ylabel('Undulator distance, m')
plt.ylim([35, 77])
plt.show()

In [ ]:
plt.plot(1e-9*gen2g.output['data']['power'][:,-1])
#plt.xlabel('Slice number')
plt.ylabel('Power, GW')
plt.xlim([0, len(gen2g.output['data']['power'])])
plt.ylim([0, np.max(1e-9*gen2g.output['data']['power'][:,-1])*1.01])
plt.title('Power with physical slices')
plt.twinx()
plt.plot(gen2g.output['data']['current'], 'orange')
plt.show()